# Imports

In [44]:
!pip install rasterio
!pip install tqdm
!pip install GDAL
!pip install git+https://github.com/VolkerH/np_obb.git
!pip install shapely
!pip install tripy
!pip install mapbox-earcut
!pip install scipy
!pip install b2sdk

  Cloning https://github.com/VolkerH/np_obb.git to /tmp/pip-req-build-v927yx9h
  Running command git clone -q https://github.com/VolkerH/np_obb.git /tmp/pip-req-build-v927yx9h
  Created wheel for np-obb: filename=np_obb-0.1.1-cp36-none-any.whl size=3743 sha256=2c1cbecb28a82ff3f5f5f4aabef9b1fbaa7f85fd8dcda8c67b708f5565db76e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-piaol8ow/wheels/38/26/ab/db34af685cba62d82ee872ec807fea37954dcf0b70fb7ff2aa
Successfully built np-obb


In [80]:
#import pickle
import bz2
import lzma
import _pickle as cPickle
import rasterio
from rasterio.windows import Window
from rasterio.mask import mask
import pandas as pd
from tqdm import tqdm
import numpy as np
from osgeo import gdal
from matplotlib import pyplot as plt
import np_obb
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import tripy
import mapbox_earcut as earcut
import scipy
import os
import shutil
import zipfile
import json
import b2sdk
import ast
import requests

In [46]:
tqdm.pandas()

In [47]:
dtm = rasterio.open("/content/drive/My Drive/datas/3d_house/DTM/DTM_BRABANT_WALLON/RELIEF_WALLONIE_MNT_2013_2014.tif")

In [48]:
dsm = rasterio.open('/content/drive/My Drive/datas/3d_house/DSM/DSM_BRABANT_WALLON/RELIEF_WALLONIE_MNS_2013_2014.tif')

# Functions definition

Function: Create a folder

In [49]:
def create_folder(path):
  os.mkdir(path)

Function: Reset the folders (delete them and create them again)

In [50]:
def reset_folder():
  folders = ['house', 'web']
  
  for folder in folders:
    
    # Remove the folder
    shutil.rmtree(folder, ignore_errors=True)

    # Create a new one
    create_folder(folder)

reset_folder()

Object: Raster2xyz, library copied and modify for the specific need of this notebook.

Source: https://pypi.org/project/raster2xyz/

In [51]:
class Raster2xyz(object):

    def __init__(self, verbose=True):
      pass

    def __getXyzData(self, raster_values, no_data):
      """
      Convert a raster to x, y, z numpy array.
      """

      try:
        y, x = np.where(raster_values != no_data)
        z = np.extract(raster_values != no_data, raster_values)

        return pd.DataFrame({
            "x": x,
            "y": y,
            "z": z
        })

      except Exception as err:
        print("Error getting XYZ data: {0}".format(err))

    def get_xyz_from_raster(self, crop_result, no_data=-9999):

      # Separate the received data
      raster, meta = crop_result

      # Transform to x, y, z
      dataframe = self.__getXyzData(raster, no_data)

      # Return the dataframe and an tuple with the offsets
      return dataframe, (meta['transform'][2], meta['transform'][5])


raster2xyz = Raster2xyz()

In [52]:
def crop_raster(raster, shape):
  """Crop the raster with a given shape (Polygon)"""

  # Crop the raster
  out_image, out_transform = rasterio.mask.mask(raster, shapes=shape, all_touched=True, crop=True)
  out_meta = raster.meta

  # Remove the third axis
  out_image = np.moveaxis(out_image.squeeze(), -1, 0)

  # Update the new tiff file metadata
  out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[0],
                 "width": out_image.shape[1],
                 "transform": out_transform })

  return out_image, out_meta

  # More info: https://gis.stackexchange.com/questions/337677/rasterio-write-masked-array-to-tiff-shape-write-and-read-indices-are-wrong

In [53]:
def save_lzma(name, data):

  with lzma.open(f"{name}.xz", "wb") as f:
    cPickle.dump(data, f)

In [54]:
def get_axis_aligned_bounding_box(iterable):
  """
  Return the 2D boundig box of a given shape.
  """

  # Math the bounding box.
  min_x, min_y = np.min(iterable[0], axis=0)
  max_x, max_y = np.max(iterable[0], axis=0)

  # Extend the bounds by 8 in every direction.
  extension = 8
  min_x -= extension
  min_y -= extension

  max_x += extension
  max_y += extension

  # Return a list of tuples with 4 vertex.
  return [(min_x, min_y), (max_x, min_y), (max_x, max_y), (min_x, max_y)]

In [55]:
def sanitize_shape(shape, bounding=True, last_is_first=False):

  if bounding:
    shape = get_axis_aligned_bounding_box([shape])
  
  if last_is_first:
    shape.append(shape[0])

  return [{'type': 'Polygon', 'coordinates': [shape]}]

In [56]:
def get_bbox_oriented(np_array, bound=0):

  x_y = pd.DataFrame({'x': np_array[:, 0], 'y': np_array[:, 1]}).to_numpy()
  
  # x_y = df[['x', 'y']].to_numpy()
  corners, _ = np_obb.get_obb_from_points(x_y, calcconvexhull=False)

  return corners

In [57]:
def get_3d_bbox_oriented(np_array, bound=0):

  corners = get_bbox_oriented(np_array)

  # Get back to dataframe
  df = pd.DataFrame({'x': corners[:, 0], 'y': corners[:, 1]})
  df2 = df.copy()

  # Add z
  df['z'] = -9999
  df2['z'] = 300

  # Merge and return
  return pd.concat([df, df2]).to_numpy()

In [58]:
def math_land_center_offset(df_land):
  """
  df: pointcloud of the land.
  """

  x_offset = (df_land['x'].min() + df_land['x'].max()) / 2
  y_offset = (df_land['y'].min() + df_land['y'].max()) / 2
  z_offset = (df_land['z'].min() + df_land['z'].max()) / 2

  return (x_offset, y_offset, z_offset)

In [59]:
def math_house_offset(offset_land, offset_house, offset_relative, dtm_house, dtm_land):
  """
  offset_relative: offset math with "math_land_center_offset".
  """

  # Math the offset of the house relative to the land.
  x_offset_relative = (- (offset_house[1] - offset_land[1])) - offset_relative[0]
  y_offset_relative = (offset_house[0] - offset_land[0]) - offset_relative[1]
  z_offset_relative = (dtm_house['y'].min() - dtm_land['y'].min()) + offset_relative[2]

  return (x_offset_relative, y_offset_relative, z_offset_relative)

In [60]:
from scipy.spatial import Delaunay

def get_shape_triangulated(shape):
  verts = np.array(shape).reshape(-1, 2)
  result = Delaunay(shape).simplices
  
  return verts[result]

In [61]:
def math_xy_point(point, offset):
  return (point[0] + offset[0], point[1] + offset[1])

Function: Convert a string list to a Python Object-list

In [62]:
def string_to_shape(string):
  return list(ast.literal_eval(string))

In [63]:
def create_files(id, shapes, metadata, details):
  
  # Transform shape into usable data for the following code
  shp_land_bounded = sanitize_shape(shapes[0], True)
  shp_land = sanitize_shape(shapes[0], False)

  shp_house = sanitize_shape(shapes[1], False)
  shp_house_triangulated = get_shape_triangulated(shapes[1])

  shp_vegetation = shp_land.copy()
  shp_vegetation[0]['coordinates'].append(shp_house[0]['coordinates'][0])
  
  data = {}


  # ---- Create Land -----

  # create DTM property bounded
  data['dtm_land_bounded'] = raster2xyz.get_xyz_from_raster(crop_raster(dtm, shp_land_bounded))[0].to_numpy()
  dtm_land, offset_land = raster2xyz.get_xyz_from_raster(crop_raster(dtm, shp_land))

  data['translation_land'] = math_land_center_offset(dtm_land)
  data['dtm_land'] = dtm_land.to_numpy()

  # Create DTM property oriented bbox
  data['dtm_land_bbox'] = get_3d_bbox_oriented(data['dtm_land']) # From dtm_land


  # ---- Create Vegetation -----

  # Create DSM property not bounded
  data['dsm_vegetation'] = raster2xyz.get_xyz_from_raster(crop_raster(dsm, shp_vegetation))[0].to_numpy()


  # ----- House Trianglulated ----- 
  houses_data = None

  try:
    # Try to create a triangulated house
    houses_data = []

    for entry in shp_house_triangulated:

      shape = sanitize_shape(entry.tolist(), False, True)
      dtm_house, offset_house = raster2xyz.get_xyz_from_raster(crop_raster(dtm, shape))

      houses_data.append({
          'dtm': dtm_house.to_numpy(),
          'dsm': raster2xyz.get_xyz_from_raster(crop_raster(dsm, shape))[0].to_numpy(),
          'translation': math_house_offset(offset_land, offset_house, data['translation_land'], dtm_house, dtm_land)
      })

  except:
      # If it fail, create a normal house

      dtm_house, offset_house = raster2xyz.get_xyz_from_raster(crop_raster(dtm, shp_house))

      houses_data = [{
          'dtm': dtm_house.to_numpy(),
          'dsm': raster2xyz.get_xyz_from_raster(crop_raster(dsm, shp_house))[0].to_numpy(),
          'translation': math_house_offset(offset_land, offset_house, data['translation_land'], dtm_house, dtm_land)
      }]

  data['houses'] = houses_data

  # ---- Add metadata ----

  data['meta'] = metadata
  data['details'] = details

  # ---- Save the file ----

  save_lzma(f"./house/house_{id}", data)

# Process

## City dataset import

In [64]:
postal_code = 1435

In [65]:
address = pd.read_csv(f"/content/drive/My Drive/datas/3d_house/cities/city_{postal_code}.csv", sep=";")

In [66]:
address.head()

,ADR_NUMERO,RUE_ID,RUE_NM,CODE_POSTAL,COM_NM,X,Y,shape_property,area_property,shape_building,area_building
0,6,7702672,Clos Emile Fabry,1435,Mont-Saint-Guibert,4.659528,50.648423,"[(170581.2706426699, 148677.21507971175), (170...",1011.5873,"[(170573.85471454787, 148685.59207473416), (17...",169.9814
1,2,7702697,Rue de Corbais,1435,Mont-Saint-Guibert,4.614342,50.637944,"[(167385.81868781263, 147509.9480282804), (167...",475.2238,"[(167366.28966235323, 147492.62907465268), (16...",447.1111
2,22,7702675,Grand'Place,1435,Mont-Saint-Guibert,4.611022,50.634580,"[(167146.26764982328, 147129.5701059159), (167...",118.0624,"[(167146.26764982328, 147129.5701059159), (167...",118.0624
3,6,7702675,Grand'Place,1435,Mont-Saint-Guibert,4.610455,50.634310,"[(167105.14670673432, 147110.53204503562), (16...",273.1282,"[(167105.14670673432, 147110.53204503562), (16...",209.3004
4,19,7702680,Place de la Dodaine,1435,Mont-Saint-Guibert,4.610321,50.640581,"[(167090.49379701656, 147799.4672572501), (167...",27.4649,"[(167090.49379701656, 147799.4672572501), (167...",27.4649


## Create the dataset of each house

In [67]:
def add_property_shape(input):
  index = input[0]
  shapes = (string_to_shape(input[1]), string_to_shape(input[2]))

  meta = {
      'street_name': input[5],
      'house_number': input[6],
      'postal_code': input[4],
      'city_name': input[3],
      'x': input[7],
      'y': input[8],
  }

  details = {
      'area_property': input[9],
      'area_building': input[10]
  }

  create_files(index, shapes, meta, details)

  return index

In [68]:
start = 0
stop = address.shape[0]
counter = pd.Series([i for i in range(start + 1, start + stop + 1)])

address['counter'] = counter

In [69]:
address['id'] = address[['counter', 'shape_property', 'shape_building', 'COM_NM', 'CODE_POSTAL', 'RUE_NM', 'ADR_NUMERO', 'X', 'Y', 'area_property', 'area_building']].progress_apply(add_property_shape, axis=1)

100%|██████████| 2288/2288 [07:10<00:00,  5.31it/s]


In [70]:
print(address['counter'])

0          1
1          2
2          3
3          4
4          5
        ... 
2283    2284
2284    2285
2285    2286
2286    2287
2287    2288
Name: counter, Length: 2288, dtype: int64


## Upload houses to backblaze

In [71]:
from b2sdk.v1 import InMemoryAccountInfo, B2Api

# Init B2 API
info = InMemoryAccountInfo()
api = b2sdk.v1.B2Api(info)

# Authorize account
application_key_id = 'bcb83f531e5f'
application_key = '003e1df59dc47b642cbbf4980c5fea980121d99299'

api.authorize_account("production", application_key_id, application_key)
bucket = api.get_bucket_by_name('wallonia-lidar')
print('Account authorized - Bucket connected')

def upload_file(local_name, remote_name):
  bucket.upload_local_file(local_file=local_name, file_name=remote_name)

Account authorized - Bucket connected


In [72]:
for root, dirs, files in os.walk('house'):
  for file in files:
    upload_file(os.path.join(root, file), f'houses/{file}')

# Create the street and number JSON

In [73]:
def create_json(df, file_name, json_name):

  # Setup the dictionnary
  dictionnary = {}

  for entry in df.values.tolist():
    dictionnary[entry[1]] = entry[0]

  with open(f'./web/{file_name}.json', 'w') as fp:
      json.dump(dictionnary, fp)

Streets JSON

In [74]:
streets = address[['RUE_ID', 'RUE_NM', 'CODE_POSTAL']].copy()

In [75]:
streets.drop_duplicates(inplace=True)
postal_codes = streets['CODE_POSTAL'].unique().tolist()


for code in postal_codes:

  current_streets = streets.loc[streets['CODE_POSTAL'] == code][['RUE_ID', 'RUE_NM']]
  create_json(current_streets, code, 'streets')


House numbers JSON

In [76]:
numbers = address[['ADR_NUMERO', 'RUE_ID', 'id', 'CODE_POSTAL']].copy()
postal_codes = streets['CODE_POSTAL'].unique().tolist()

for code in postal_codes:

  streets_id = numbers[['RUE_ID', 'CODE_POSTAL']].copy()
  streets_id = streets_id.drop_duplicates()
  streets_id = streets_id.loc[streets_id['CODE_POSTAL'] == code][['RUE_ID']].values.tolist()

  create_folder(f'./web/{code}')

  for street in streets_id:
    df = numbers.loc[numbers['RUE_ID'] == street[0]]
    df = df[['id', 'ADR_NUMERO']]
  
    create_json(df, f'{postal_code}/{street[0]}', 'numbers')

#create_zip('web', 'web')

Postal Codes JSON

In [92]:
response = requests.get(url='https://static.wallonia.ml/file/wallonia-lidar/web/postal_codes.json')

postal_codes_file = None

if response.status_code == 200:
  postal_codes_file = response.json()

else:
  postal_codes_file = {}

for code in postal_codes:
  postal_codes_file[str(code)] = code

with open('web/postal_codes.json', 'w') as json_file:
  json.dump(postal_codes_file, json_file)

{'1435': 1435}


In [85]:
for root, dirs, files in os.walk('web'):
  for file in files:
    upload_file(os.path.join(root, file), os.path.join(root, file))

# Debug code

In [ ]:


"""shapes = (string_to_shape(address.loc[0]['shape_property']),
          string_to_shape(address.loc[0]['shape_building']))

points = create_files(0, shapes, 0, 0).to_numpy()
